# Homogeneisation - Visco-elastic materials

### TODO : Description du notebook 

---

## I- Importation des classes et des modules utiles

### A exécuter si notebooks.ai

In [ ]:
!pip3 install ipywidgets # Installation du package permettant la gestion des widgets
clear_output()
print("Package downloaded")
from IPython.display import clear_output, display, Markdown
import ipywidgets as widgets
from classes_v2 import *
from os import listdir
import pandas as pd

print("Modèles importés : {}".format(list_models))

### A exécuter si local

In [1]:
from IPython.display import clear_output, display, Markdown
import ipywidgets as widgets
from classes_v2 import *
from os import listdir
import pandas as pd
import matplotlib

clear_output()
print("Package downloaded")
%matplotlib widget
print("Modèles importés : {}".format(list_models))

Package downloaded
Modèles importés : [<class 'classes_v2.Mori_Tanaka'>, <class 'classes_v2.Differential_Scheme'>, <class 'classes_v2.Autocoherent_Hill'>, <class 'classes_v2.Autocoherent_III'>, <class 'classes_v2.Autocoherent_IV'>]


--- 

## II- Fonctions utiles

In [ ]:
parameters_name = {
    'K': 'Bulk modulus K',
    'G': 'Shear modulus G',
    'E': 'Young modulus E',
    'nu': "Poisson's ratio " + r'$\nu$',
    "G'": "Storage shear modulus G'",
    "G''": "Loss shear modulus G''",
    "K'": "Storage bulk modulus K'",
    "K''": "Loss bulk modulus K''",
    "E'": "Storage Younk modulus E'",
    "E''": "Loss Younk modulus E''"
}
parameters_name_bis = {value: key for (key, value) in parameters_name.items()}
input_folder = "inputs/viscoelastic_behaviors" # Dossier contenant les comportements visco-élastiques

def gen_tab_behavior():
    """
    Routine qui génère un widget de type 'tab' permettant le choix des paramètres associés aux comportements. 
    Chaque onglet généré correspond à un comportement du dictionnaire 'dict_behaviors' implémenté dans 'classes.py'.
    La fonction renvoie :
    - une liste de liste de widgets (list_widgets) contenant autant de listes que d'onglet. Chaque liste contient les widgets générant les paramètres asssociés au comportement 
    de l'onglet correspondant.
    - un widget tab correspondant au résultat final à afficher.    
    """
    behaviors_str = list(dict_behaviors_visco.keys()) # Liste des noms des comportements déjà implémentés, définis dans classes.py
    list_widgets = [] # Liste de liste, chaque liste correspond à un onglet du tab et contient les widgets de cet onglet non formatés
    tab_titles = [] # Nom des onglets, chaque onglet est associé à un type de comportement
    # Construction des widgets associés à chaque onglet
    for behavior_str in behaviors_str:
        widgets_onglet = []
        parameters = dict_behaviors_visco[behavior_str] # Paramètres associés au comportement (exemple : ['K', 'G'] pour Isotropic)
        for parameter in parameters:
            w = widgets.BoundedFloatText(value=200, min=0, max=10**12, step=1) # Widget associé au paramètre parameter
            if parameter == 'nu':
                w.max = 0.5
                w.value = 0.3
                w.step = 0.1
            # Traitement des paramètres visco-élastiques
            if parameter.endswith("'"):
                # TODO: expliquer le format attendu
                # Récupération de la liste des fichiers de comportement disponibles
                behavior_files = listdir(input_folder)
                # Sélection des fichiers .txt et .csv seuls
                behavior_files = [file for file in behavior_files if (file.endswith('.txt') or file.endswith('.csv'))]
                # Création du widget
                w = widgets.Dropdown(options=behavior_files)
            w_label = widgets.Label(value=parameters_name[parameter])
            widgets_onglet.append(widgets.HBox([w_label, w]))
        # Ajout d'une description des fichiers d'entrée et des boutons permettant de choisir l'abscisse (température ou fréquence)
        if 'Visco-elastic' in behavior_str:
            w_label = widgets.Label(value="Entry file must have two rows separated with a ',': one for the temperature/frequency, one for the values of the loss/storage modulus.\nPlease, select the nature of the abscissa.")
            w_choice = widgets.Dropdown(options=["temperature", "frequency"])
            widgets_onglet.append(widgets.VBox([w_label, w_choice]))
        list_widgets.append(widgets_onglet)
        tab_titles.append(behavior_str)
    # Création du tab
    tab = widgets.Tab()
    tab.children = [widgets.VBox(w) for w in list_widgets]
    for pos, title in enumerate(tab_titles):
        tab.set_title(pos, title)
    return list_widgets, tab

def read_behavior(tab, list_widgets):
    """
    Fonction qui, à partir d'un widget 'tab' comme celui construit par la fonction précédente, renvoie le dictionnaire 'behavior' construit par l'utilisateur.
    Dans le cas d'un comportement viso-élastique, renvoie une liste de fréquences/températures notée frequency.
    Les valeurs du dict behavior sont alors des listes.
    """
    global input_folder
    behavior_int = tab.selected_index # Onglet ouvert par l'utilisateur
    behavior_str = tab.get_title(behavior_int)
    widgets_parameters = list_widgets[behavior_int] # Widgets des paramètres de l'onglet ouvert
    if 'isotropic' in behavior_str:
        # Cas élastique classique
        frequency = [] # Les paramètres sont indépendants de la fréquence/température
        abscissa = "frequency"
        behavior = {parameters_name_bis[w.children[0].value]: w.children[1].value for w in widgets_parameters}
    else:
        # Cas visco-élastique
        behavior_visco = {} # Variable intermédiaire
        behavior = {}
        for w in widgets_parameters[:-1]:
            parameter_name = parameters_name_bis[w.children[0].value] # Lecture du nom du paramètre (ex: 'K', "G'")
            if not parameter_name.endswith("'"):
                # Cas élastique
                behavior[parameter_name] = w.children[1].value # Valeur du widget FloatSlider associé
            else:
                filename = input_folder + '/' + w.children[1].value # Fichier choisi par l'utilisateur
                # Lecture du fichier
                parameter_values = [] # Liste de tuples (frequency, value)
                with open(filename, 'r') as file:
                    lines = file.readlines()
                    for line in lines:
                        line = line.strip().split()
                        line = [float(element) for element in line]
                        f, value = line
                        parameter_values.append((f, value))
                # Tri des valeurs par fréquence croissante
                parameter_values.sort()
                # Extraction des listes frequency et behavior
                frequency = [element[0] for element in parameter_values]
                behavior_visco[parameter_name] = [element[1] for element in parameter_values]
        # Transformation des paramètres élastiques en liste de valeurs de la même taille que frequency
        for parameter, value in behavior.items():
            behavior[parameter] = np.array(len(frequency)*[value])
        # Construction des paramètres complexes
        for parameter, values in behavior_visco.items():
            if parameter.endswith("''"):
                parameter_complex = parameter[:-2] # Nom du paramètre complexe, exemple 'G' pour "G''"
                behavior[parameter_complex] = np.array(behavior_visco[parameter[:-1]]) + 1j*np.array(behavior_visco[parameter])
        # lecture de la nature de l'abscisse (température ou fréquence)
        w_abscissa = widgets_parameters[-1]
        w_choice = w_abscissa.children[1]
        abscissa = w_choice.value
    return behavior, frequency, abscissa

def gen_tab_type():
    """
    Routine qui génère un widget de type 'tab' permettant le choix des paramètres associés aux types d'inclusions (aspect ratio, orientation, etc..). 
    TODO : Inclure l'orientation
    Chaque onglet généré correspond à un type du dictionnaire 'dict_types' implémenté dans 'classes.py'.
    La fonction renvoie :
    - une liste de liste de widgets (list_widgets) contenant autant de listes que d'onglets. Chaque liste contient les widgets générant les paramètres asssociés au type 
    de l'onglet correspondant.
    - un widget tab correspondant au résultat final à afficher.    
    """
    list_widgets = [] # Liste de listes, chaque liste correspond à un onglet et contient les widgets de cet onglet
    # Construction des widgets associés à chaque onglet
    for type_int in dict_types.keys():
        if type_int == 0:
            # Sphères, pas besoin de paramètres supllémentaires
            list_widgets.append([])
        else:
            # Initialisation
            list_widgets.append([])
            # Ellipsoïdes
            for n in ['1', '2']:
                w_label = widgets.Label(value="Aspect ratio " + str(n))
                w_aspect_ratio = widgets.BoundedFloatText(value=0.5, min=0.1, max=10, step=0.5)
                list_widgets[-1] += [widgets.HBox([w_label, w_aspect_ratio])]
    # Création du tab
    tab = widgets.Tab()
    # Attribution des onglets
    tab.children = [widgets.HBox(w) for w in list_widgets]
    # Attribution des noms des onglets
    for pos, title in dict_types.items():
        tab.set_title(pos, title)
    return list_widgets, tab         

def read_type(tab, list_widgets):
    """
    Fonction qui, à partir d'un widget 'tab' comme celui construit par la fonction gen_tab_type, renvoie le type d'inclusion et le rapport d'apsect.
    """
    type_int = tab.selected_index # Onglet ouvert par l'utilisateur
    tab_name = tab.get_title(type_int) # Active tab name
    if tab_name=='Spheres':
        # Spheres
        aspect_ratio = (1,1)
    elif tab_name=='Ellipsoids':
        # Ellipsoids
        c1 = list_widgets[type_int][0].children[1].value
        c2 = list_widgets[type_int][1].children[1].value
        aspect_ratio = (c1, c2)
    return type_int, aspect_ratio

def str_to_model(model_name):
    """
    Fonction qui renvoie l'instance de classe Model associée au nom de classe model_name (str).
    """
    for Model in list_models:
        model = Model()
        if model.name.upper() == model_name.upper():
            return model
        
def incr(value, value_incr, mini, maxi):
    """
    Incrémente la valeur de la variable value si celle-ci ne dépasse pas les bornes maxi et mini. 
    Renvoie un booléen qui indique si la variable a été incrémentée ou non (c'est le cas lorsque la variable est en dehors des bornes min max)
    """
    result = value
    result += value_incr
    if result > maxi:
        result = maxi
    if result < mini:
        result = mini
    changed = (result != value)
    return result, changed

---

## III- Calcul de comportement homogénéisé de microstructures
Cette section permet la génération manuelle de microstructure et le calcul de comportement homogénéisé avec les modèles disponibles.

In [ ]:
# Initialisation du graphe de tracé des modèles
plotted = {} # Dictionnaire des modèles déjà tracés, format {model: behavior}
fig = None # Figure sur laquelle seront tracés les modèles
matplotlib.rcParams.update({'font.size': 6}) # Changement de la taille de la police

dict_inclusions = {}
# Initialise la liste des inclusions créees. dict_inclusions est sous la forme {name_inclusion (str): inclusion (Inclusion)}

# Nom de l'inclusion
w_label = widgets.Label(value='Inclusion name')
n_inclusion = 0 # Identifiant servant à donner automatiquement un nom unique à chaque inclusion
w_name = widgets.Text(value='inclusion'+str(n_inclusion))

# Type d'inclusion
widgets_type, tab_type = gen_tab_type() # Génération du widgets tab de choix du type d'inclusion

# Comportement de l'inclusion
caption = widgets.Label(value='Inclusion behavior')
list_widgets, tab = gen_tab_behavior()

# Génération de l'inclusion
button_generate_inclusion = widgets.Button(description="Generate Inclusion")
output = widgets.Output()
def generate_inclusion(b):
    """
    Fonction appelée lors d'un click sur le bouton, génère une inclusion avec les paramètres choisis.
    """
    global n_inclusion
    # Récupération des paramètres choisis
    output.clear_output()
    inclusion_name = w_name.value
    if inclusion_name in list(dict_inclusions.keys()):
        with output:
            print("Name already exists")
    else :
        type_inclusion, inclusion_aspect_ratio = read_type(tab_type, widgets_type)
        behavior, frequency, abscissa = read_behavior(tab, list_widgets)
        inclusion = Inclusion(
            type_inclusion,
            behavior,
            name=inclusion_name,
            aspect_ratio=inclusion_aspect_ratio,
            frequency=frequency,
            abscissa=abscissa
        )
        dict_inclusions[inclusion_name] = inclusion
        with output:
            print("Inclusion generated: ", inclusion)
            inclusion.graph_parameter()
        # Mise à jour automatique du nom de l'inclusion
        n_inclusion += 1
        w_name.value = 'inclusion'+str(n_inclusion)
    
button_generate_inclusion.on_click(generate_inclusion)

# Génération d'inclusions avec interphase
# Nom de l'inclusion
w_label_bis = widgets.Label(value='Instance name')
n_inclusion_bis = 0 # Identifiant servant à donner automatiquement un nom unique à chaque inclusion
w_name_bis = widgets.Text(value='instance'+str(n_inclusion_bis))

# Type d'inclusion
widgets_type_bis, tab_type_bis = gen_tab_type() # Génération du widgets tab de choix du type d'inclusion

# Comportement de l'inclusion
caption_incl = widgets.Label(value='Inclusion behavior')
list_widgets_incl, tab_incl = gen_tab_behavior()
# Comportement de l'interphase
caption_inter = widgets.Label(value='Interphase behavior')
list_widgets_inter, tab_inter = gen_tab_behavior()

# Génération de l'inclusion
button_generate_inclusion_bis = widgets.Button(description="Generate inclusion with interphase", layout={'width': 'max-content'})
output_bis = widgets.Output()

def generate_inclusion_bis(b):
    """
    Fonction appelée lors d'un click sur le bouton, génère une inclusion avec interphase avec les paramètres choisis.
    """
    global n_inclusion_bis
    # Récupération des paramètres choisis
    output_bis.clear_output()
    instance_name = w_name_bis.value
    if instance_name in list(dict_inclusions.keys()):
        with output_bis:
            print("Name already exists")
    else :
        type_inclusion, inclusion_aspect_ratio = read_type(tab_type_bis, widgets_type_bis)
        behavior_incl, frequency_incl, abscissa_incl = read_behavior(tab_incl, list_widgets_incl) # behavior of the inclusion
        behavior_inter, frequency_inter, abscissa_inter = read_behavior(tab_inter, list_widgets_inter) # behavior of the interphase
        inclusion = Inclusion(type_inclusion,
                              behavior_incl,
                              name=instance_name+'_inclusion',
                              aspect_ratio=inclusion_aspect_ratio, 
                              frequency=frequency_incl,
                              abscissa=abscissa_incl)
        interphase = Inclusion(type_inclusion,
                               behavior_inter,
                               name=instance_name+'_interphase',
                               aspect_ratio=inclusion_aspect_ratio,
                               frequency=frequency_inter,
                               abscissa=abscissa_inter)
        instance = InclusionAndInterphase(inclusion, interphase, name=instance_name)
        dict_inclusions[instance_name] = instance
        with output_bis:
            print("Inclusion generated: ", instance)
        # Mise à jour automatique du nom de l'inclusion
        n_inclusion_bis += 1
        w_name_bis.value = 'instance'+str(n_inclusion_bis)
    
button_generate_inclusion_bis.on_click(generate_inclusion_bis)

### Construction d'une microstructure

# Création de la variable contenant la microstructure
microstructure = None # Initialisation

# Fonctions liées aux appuis sur les boutons
def add_inclusion_to_structure(b):
    """
    Fonction appelée lors d'un click sur le bouton "Add inclusion", génère un widget associé à la fraction volumique de l'inclusion et l'ajoute au dictionnaire "widgets_f".
    Crée aussi un bouton permettant la suppression de l'inclusion et l'ajoute au dictionnaire buttons.
    Enfin, affiche la ligne de widgets correspondante.
    """
    out2.clear_output()
    inclusion = dict_inclusions[w_inclusions.value]
    if inclusion in list(widgets_f.keys()):
        with out2:
            print("Already added")
    else:
        w_name = widgets.Label(inclusion.name)
        w_b = widgets.Button(description="Remove inclusion")
        w_b.on_click(remove_inclusion)
        buttons_suppress[w_b] = inclusion
        # Cas inclusion avec interphase
        if type(inclusion)==InclusionAndInterphase:
            w_f_incl = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f inclusion')
            w_f_inter = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f interphase')
            widgets_f[inclusion] = (w_name, w_f_incl, w_f_inter)
            with out1:
                display(w_name, widgets.HBox([w_f_incl, w_f_inter, w_b]))
        # Cas inclusion simple
        else:
            w_f = widgets.FloatSlider(min=0.01, max=0.99, step=0.01, description='f')
            widgets_f[inclusion] = (w_name, w_f)
            with out1:
                display(w_name, widgets.HBox([w_f, w_b]))

def add_inclusion_to_list(b):
    """
    Fonction appelée lors d'un clic sur le bouton 'generate inclusion' de la section précédente.
    met à jour le widget permettant le choix des inclusions à ajouter.
    """
    w_inclusions.options = list(dict_inclusions.keys())
    w_inclusions_info.options = list(dict_inclusions.keys())
    
def remove_inclusion(b):
    """
    Fonction appelée lorsqu'un bouton 'Remove inclusion' est appelé. 
    Repère l'inclusion associée au bouton, ferme les widgets associés et supprime l'entrée du dictionnaire widgets_f
    """
    out2.clear_output()
    inclusion = buttons_suppress[b]
    # Fermeture des widgets
    b.close()
    for w in widgets_f[inclusion]:
        w.close()
    del widgets_f[inclusion]
    del buttons_suppress[b]

def generate_microstructure(b):
    """
    Génère la microstructure avec les paramètres choisis par l'utilisateur. 
    Si les fractions volumiques choisies ne sont pas cohérentes, affiche un message.
    Affiche une description de la microsructure créee.
    """
    global microstructure
    reset_graph(None)
    matrix_behavior, frequency, abscissa = read_behavior(tab_m, widgets_m) # Lecture des widgets définis dans la section 'Comportement de la matrice'
    dict_inclusions = {}
    # Lecture des fractions volumiques choisies
    for inclusion, widgets in widgets_f.items():
        # Inclusions with an interphase
        if type(inclusion)==InclusionAndInterphase:
            w_name, w_f_incl, w_f_inter = widgets
            f = (w_f_incl.value, w_f_inter.value)
        else:
            w_name, w_f = widgets
            f = w_f.value
        dict_inclusions[inclusion] = f
    # Génération de la microstructure
    out3.clear_output()
    out4.clear_output()
    try:
        microstructure = Microstructure(matrix_behavior, dict_inclusions, frequency=frequency, abscissa=abscissa)
        with out3:
            print("Microstructure generated\n" + str(microstructure))
            # Dessin de la microstructure
            microstructure.draw()
        with out4:
            # Graphe de comportement visco-élastique
            microstructure.graph_parameter()
    except NameError:
        microstructure = None
        with out3:
            print("Inconsistent choice of volume fractions")
            
### Affichage des widgets inclusion
# Inclusion simpledisplay(Markdown("## Inclusion generation"))
display(Markdown("## Inclusion generation"))
w_inclusion = widgets.VBox([w_label, w_name, widgets.Label(value='Inclusion type'), tab_type, caption, tab, button_generate_inclusion, output],
                          layout={'border': '1px solid #FF625BF5'})
display(w_inclusion)
# Inclusion + Interphase
display(Markdown("## Inclusion and interphase generation"))
w_inclusion_bis = widgets.VBox([w_label_bis,
                                w_name_bis,
                                widgets.Label(value='Inclusion type'),
                                tab_type_bis,
                                caption_incl,
                                tab_incl,
                                caption_inter,
                                tab_inter,
                                button_generate_inclusion_bis,
                                output_bis],
                          layout={'border': '1px solid #FF625BF5'})
display(w_inclusion_bis)

### Visualisation et suppression d'inclusions
w_label = widgets.Label(value="Display info on the generated inclusions")
w_inclusions_info = widgets.Dropdown(options=list(dict_inclusions.keys()), layout={'width': 'max-content'})
w_delete = widgets.Button(description="Delete inclusion", layout={'width': 'max-content'})
out_inclusions_info = widgets.Output()
# Affichage des infos de l'inclusion selectionnée
def display_info(change):
    """
    Fonction appelée lors d'un changement de value du widget w_inclusions_info.
    Récupère l'inclusion choisie et affiche sa description.
    """
    out_inclusions_info.clear_output()
    try:
        inclusion = dict_inclusions[w_inclusions_info.value]
    except KeyError:
        inclusion = None
    with out_inclusions_info:
        print(inclusion)
        
w_inclusions_info.observe(display_info, names='value')
# Suppression d'inclusions
def delete_inclusion(b):
    """
    Fonction appelée lors d'un appui sur le bouton 'Delete inclusion'.
    Récupère l'inclusion sélectionnée et la supprime de la bibliothèque d'inclusions.
    """
    inclusion_name = w_inclusions_info.value
    try:
        del dict_inclusions[inclusion_name]
    except KeyError:
        None
    w_inclusions.options = list(dict_inclusions.keys())
    w_inclusions_info.options = list(dict_inclusions.keys())

w_delete.on_click(delete_inclusion)
# Affichage des widgets infos inclusions
display(Markdown("## Inclusions info"))
display(widgets.VBox([w_label, widgets.HBox([w_inclusions_info, w_delete]), out_inclusions_info], layout={'border': '1px solid #FF625BF5'}))

# Comportement de la matrice
caption = widgets.Label(value='Matrix behavior')
widgets_m, tab_m = gen_tab_behavior()
out4 = widgets.Output() # Dessin du graphe de comportement visco-élastique

# Ajout d'inclusions
w_inclusions = widgets.Dropdown(options=list(dict_inclusions.values()), layout={'width': 'max-content'})
button_add_inclusion = widgets.Button(description="Add inclusion")
out1 = widgets.Output()
out2 = widgets.Output()
widgets_f = {} # Dictionnaire des inclusions ajoutées et de leurs widgets ('name','fraction volumique') associés
buttons_suppress = {} # Dictionnaire des boutons permettant de supprimer une inclusion de la structure et inclusions associés

button_add_inclusion.on_click(add_inclusion_to_structure)
button_generate_inclusion.on_click(add_inclusion_to_list)
button_generate_inclusion_bis.on_click(add_inclusion_to_list)

# Génération de la microstructure
b_generate_structure = widgets.Button(description='Generate microstructure', layout={'width': 'max-content'})
# TODO : widget 'valid' qui indique en temps réel si les fractions volumiques choisies sont cohérentes
out3 = widgets.Output() # Dessin du VER
b_generate_structure.on_click(generate_microstructure)

# Affichage des widgets microstructure
display(Markdown("## Microstructure generation"))
w_micro = widgets.VBox([caption, tab_m, widgets.HBox([w_inclusions, button_add_inclusion, out2]), out1, b_generate_structure, out4, out3],
                      layout={'border': '1px solid #FF625BF5'})
display(w_micro)

# Choix du modèle
def test_models(b=None):
    """
    Fonction appelée lors d'un appui sur le bouton 'Generate microstructure' juste en haut.
    Teste les modèles disponibles sur la microstructure générée et met à jour la liste des modèles valides 'valid_models'.
    """
    valid_models = []
    if microstructure == None:
        # Vérifie que la microstructure a bien été créee
        return None
    for Model in list_models:
        model = Model()
        valid = model.check_hypothesis(microstructure)
        if valid:
            # La microstructure vérifie les hypothèses du modèle.
            valid_models.append((model.name, model))
    # Mise à jour du widget de séléection du modèle
    select_model.options = valid_models

valid_models = [] # Liste des modèles pouvant s'appliquer à la microstructure donnée, format : [(model_name, Model)]
select_model = widgets.Dropdown()
test_models()
label = widgets.Label(value="Select a model. Only compatible models are displayed.")
b_compute = widgets.Button(description='Compute behavior')
output_behavior = widgets.Output() # Affichage des résultats du modèle


def draw_figure(frequency, abscissa):
    """
    Trace les graphes associés aux modèles du dictionnaire plotted.
    Trace un graphe par paramètre, avec les parties réelle et imaginaire de chaque modèle du dictionnaire.
    """
    global fig, plotted
    plt.close('all')
    # Création de la liste des paramètres
    parameters = ['K', 'G', 'E', 'nu']
    fig = plt.subplots(2, 2, figsize=(8 ,8))
    fig = fig[0]
    # Tracé des modèles
    for index, parameter in enumerate(parameters):
        plt.subplot(2, 2, index+1)
        for model, behavior in plotted.items():
            values = np.array(behavior[parameter])
            label = model.name + " " + parameter
            # Cas élastique
            if len(values) == 1:
                plt.plot(frequency, values.real, '.', label=label+"'")
                plt.plot(frequency, values.imag, '.', label=label+"''")
            # Cas visco-élastique
            elif abscissa == "temperature":
                plt.semilogy(frequency, values.real, label=label+"'")
                plt.semilogy(frequency, values.imag, label=label+"''")
                plt.xlim(min(frequency), max(frequency))
            elif abscissa == "frequency":
                plt.loglog(frequency, values.real, label=label+"'")
                plt.loglog(frequency, values.imag, label=label+"''")
                plt.xlim(min(frequency), max(frequency))
        # Titres des graphes et légendes des axes
        plt.title(parameters_name[parameter])
        plt.ylabel("Parameters values")
        plt.xlabel(abscissa.capitalize())
        plt.legend()
    plt.show()        
    
def reset_graph(b):
    """
    Reset la liste des modèles tracés.
    """
    global fig, plotted
    plotted = {}
    plt.close('all')
    output_behavior.clear_output()
    
    
def compute_model(b):
    """
    Fonction appelée lors d'un appui sur le bouton 'Compute behavior'.
    Récupère le modèle choisi par l'utilisateur.
    Construit une liste de fréquences communes à la matrice et aux microstructures.
    Construit de nouvelles listes de valeurs de paramètres par interpolation sur cette nouvelle liste.
    Calcule le comportement homogénéisé de la structure et l'affiche sur un graphe.
    Génère un widget permettant la sauvegarde du graphe en .pdf (ou autre format image) ou des données en .csv (ou .txt).
    """
    global fig, plotted
    h_behavior = {} # Initialisation
    ### Construction d'une liste de fréquences commune aux instances
    # Création de la liste des instances (séparation des inclusions et des interphases en instances différentes)
    instances = []
    for instance in microstructure.dict_inclusions.keys():
        if type(instance)==InclusionAndInterphase:
            instances += [instance.inclusion, instance.interphase]
        else:
            instances.append(instance)
    # Lecture des listes de fréquences non vides de la matrice et des inclusions
    frequencies = [instance.frequency for instance in [microstructure]+instances if len(instance.frequency)>1]
    # Récupération de l'abscisse
    if len(microstructure.frequency) == 0:
        # Microstructure élastique
        try:
            inclusion = list(microstructure.dict_inclusions.keys())[0]
            if type(inclusion)==InclusionAndInterphase:
                microstructure.abscissa = inclusion.inclusion.abscissa
            else:
                microstructure.abscissa = inclusion.abscissa
        except:
            # Matrice sans inclusions
            None
    # Récupération des maximums et minimums de chaque liste
    maximums = [max(list_) for list_ in frequencies]
    minimums = [min(list_) for list_ in frequencies]
    # Récupération du plus petit maximum et du plus grand minimum
    try:
        maxi, mini = min(maximums), max(minimums)
        # Construction du domaine d'interpolation
        frequency_inter = np.linspace(mini, maxi, 200)
    except ValueError:
        # Les listes sont vides, aucun des matériaux n'est visco-élastique
        frequency_inter = [0]
    # Interpolation des valeurs des paramètres de la matrice et des inclusions
    for element in [microstructure] + instances:
        if not list(element.frequency):
            # Inclusion ou matrice entièrement élastique
            for parameter, values in element.behavior.items():
                element.behavior[parameter] = [values]*len(frequency_inter)
        else:
            for parameter, values in element.behavior.items():
                element.behavior[parameter] = np.interp(frequency_inter,
                                                                element.frequency,
                                                                element.behavior[parameter])
        element.frequency = frequency_inter
    ### Calcul du comportement homogénéisé
    model = select_model.value
    # Vérification de la liste des modèles déjà tracés
    if model in plotted.keys():
        None
    else:
        homogenised_behavior = model.compute_h_behavior(microstructure)
        plotted[model] = homogenised_behavior
    ### Tracé des graphes
    abscissa = microstructure.abscissa
    output_behavior.clear_output()
    with output_behavior:
        draw_figure(frequency_inter, abscissa)
    

b_generate_structure.on_click(test_models)
b_compute.on_click(compute_model)

# Reset du graphe
b_reset = widgets.Button(description="Reset graph")
b_reset.on_click(reset_graph)

# Sauvegarde de la figure en pdf
w_figname_pdf = widgets.Text(value="fig.pdf")
b_save_pdf = widgets.Button(description="Save figure as pdf/image", layout={'width': 'max-content'})
out_savepdf = widgets.Output()
w_pdf = widgets.HBox([w_figname_pdf, b_save_pdf, out_savepdf])

def save_pdf(b):
    """
    Fonction appelée lors de l'appui sur le bouton save figure as pdf.
    Capture le nom (avec extension) donné par l'utilisateur et sauvegarde la figure dans le dossier ouputs/visco-elasticity.
    """
    global fig
    folder = "outputs/visco-elasticity/"
    file_name = w_figname_pdf.value
    # Vérification de l'extension
    if '.' in file_name and not file_name.endswith('.'):
        try:
            fig.savefig(folder+file_name)
            with out_savepdf:
                print("Done!")
        except:
            with out_savepdf:
                print("Extension error")
    else:
        with out_savepdf:
            print("No extension detected")
    
b_save_pdf.on_click(save_pdf)

# Sauvegarde de la figure en csv
w_figname_csv = widgets.Text(value="fig")
b_save_csv = widgets.Button(description="Save figure as csv", layout={'width': 'max-content'})
out_savecsv = widgets.Output()
w_csv = widgets.HBox([w_figname_csv, b_save_csv, out_savecsv])

def save_csv(b):
    """
    Fonction appelée lors de l'appui sur le bouton save figure as csv.
    Capture le nom (avec extension) donné par l'utilisateur et sauvegarde la figure dans le dossier ouputs/visco-elasticity.
    """
    global plotted
    folder = "outputs/visco-elasticity/"
    # Récupération du nom du fichier de sortie, sans extension
    file_name = w_figname_csv.value
    ### Création du dictionnaire contenant les données du dataframe
    # Initialisation de data
    data = {microstructure.abscissa.capitalize(): microstructure.frequency}
    # Ajout des données des modèles tracés
    for model, behavior in plotted.items():
        for parameter, values in behavior.items():
            entry1 = model.name + ' - ' + parameter + "'" # Partie réelle
            entry2 = entry1 + "'" # Partie imaginaire
            data[entry1] = np.array(values).real
            data[entry2] = np.array(values).imag
    # Enregistrement
    pd.DataFrame(data).to_csv(folder+file_name+'.csv', header=True, index=False, sep=',', mode='w')
    
b_save_csv.on_click(save_csv)

# Affichage des widgets model
w_models = widgets.VBox([label, widgets.HBox([select_model, b_compute]), output_behavior, b_reset, w_pdf, w_csv],
                       layout={'border': '1px solid #FF625BF5'})
display(Markdown("## Available models"))
display(w_models)

---